# DICE model
This is a simplified version of the DICE model implemented in Python.

The Dynamic Integrated Climate-Economy model, referred to as the DICE model, is a computer-based integrated assessment model that “integrates in an end-to-end fashion the economics, carbon cycle, climate science, and impacts in a highly aggregated model that allows a weighing of the costs and benefits of taking steps to slow greenhouse warming."

<left><img src="util/simple_DICE_diagram.png" width="700px">

#### Import libraries

In [1]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from ipywidgets import widgets

from util.simple_DICE import simple_DICE

#### Simulation time-range and time-step

In [2]:
time_step = 5 # years. If the time step (5 years) is changed then some of the equations below will need to be adapted 
time_range = np.arange(2015,2115,5)

#### Function to update the simulation when changing the parameters with the sliders

In [3]:
def update_sim(pop_param, prod_grow_rate, ec_grow_rate, temp_CO2_factor, clim_damage_exp):
    
    model = simple_DICE(pop_param.value, 
                        prod_grow_rate.value, 
                        ec_grow_rate.value, 
                        CO2_temp_factor.value, 
                        clim_damage_exp.value)
    
    population, capital, prod, gross_output, sigma, backstop_price, ec_rate, ec_cost_coeff, ec_frac, CO2_emis, CO2_conc, temp, clim_damage_frac = model.simulation()
    ec_cost = ec_frac*gross_output
    clim_damage = clim_damage_frac*gross_output
    rel_clim_damage = (ec_cost+clim_damage)/gross_output *100
    
    return population, prod, ec_cost, temp, rel_clim_damage

#### Function to update the figure when changing the parameters with the sliders

In [4]:
def update_figure(change):
    with fig.batch_animate(duration=10):
        fig.data[2].y = update_sim(pop_param, prod_grow_rate, ec_grow_rate, CO2_temp_factor, clim_damage_exp)[0]
        fig.data[1].y = update_sim(pop_param, prod_grow_rate, ec_grow_rate, CO2_temp_factor, clim_damage_exp)[3]
        fig.data[0].y = update_sim(pop_param, prod_grow_rate, ec_grow_rate, CO2_temp_factor, clim_damage_exp)[4]

## World population
Population starts at 7.4 billion people in the year 2015 and grows according to a net growth rate that steadily declines until it reaches 12 billion, at which point the population stabilizes. The declining rate of growth means that as time goes on, the rate of growth decreases, so we approach 12 billion very gradually. 

In [5]:
# Population slider
pop_param = widgets.FloatSlider(min = 0.00,
                                 max = 1,
                                 value = 0.13, step = 0.01,
                                 description = 'Population growth factor',
                                 continuous_update=True,
                                 style = {'description_width': '175px'} ,layout={'width': '175px'},
                                 orientation='vertical')
pop_param.observe(update_figure,names = 'value')

## Global capital
In this model, Global Capital is a reservoir that represents all the goods and services of the global economic system; so this is much more than just money in the bank. This reservoir increases as a function of investments and decreases due to depreciation. Depreciation means that value is lost, and the model assumes a 10% depreciation per year; the 10% value comes from observations of rates of depreciation across the global economy in the past.

The savings rate is 18.5% per year (again based on observations). The Gross Output is the total economic output for each year, which depends on the global population, a productivity factor, and Global Capital.

## Factor of productivity
The model assumes that our economic productivity will increase due to technological improvements, but the rate of increase will decrease, just like the rate of population growth.  So the productivity keeps increasing, but it does not accelerate, which would lead to exponential growth in productivity. This decline in the rate of technological advances is once again something that is based on observations from the past.

In [6]:
# Productivity slider
prod_grow_rate = widgets.FloatSlider(min = 0.00,
                                 max = 0.2,
                                 value = 0.08, step = 0.01,
                                 description = 'Productivity growth rate',
                                 continuous_update=True,
                                 style = {'description_width': '175px'} ,layout={'width': '175px'},
                                 orientation='vertical')
prod_grow_rate.observe(update_figure,names = 'value')

## Gross outputs or Gross production
The Gross Output is the total economic output for each year, which depends on the global population, the factor of productivity and global capital.

## Emissions Control (EC)
The Emissions Control is the set of measures aiming to reduce carbon emissions. If we do nothing, its cost is zero, but as we try to do more and more in terms of emissions reductions, the EC costs go up. If we go all out in this department, the EC Costs can rise to be 15% of the Gross Output.

In [7]:
# Emissions control slider
ec_grow_rate = widgets.FloatSlider(min = 0,
                                 max = 0.12,
                                 value = 0.08, step = 0.01,
                                 description = 'Emissions control growth rate',
                                 continuous_update=True,
                                 style = {'description_width': '175px'} ,layout={'width': '175px'},
                                 orientation='vertical')
ec_grow_rate.observe(update_figure,names = 'value')

## CO2
The model calculates the carbon emissions as a function of the Gross Output of the global economy and two adjustable parameters, one of which (sigma) sets the emissions per dollar value of the Gross Output (units are in metric tons of carbon per trillion dollars of Gross Output) and something called the Emissions Control Rate (ECR).

Currently, sigma has a value of about 0.118, and the model we will use assumes that this will decrease as time goes on due to improvements in efficiency of our economy — we will use less carbon to generate a dollar’s worth of goods and services in the future, reflecting what has happened in the recent past. The ECR can vary from 0 to 1, with 0 reflecting a policy of doing nothing with respect to reducing emissions, and 1 reflecting a policy where we do the maximum possible. Note that when ECR = 1, then the whole Emissions equation above gives a result of 0 — that is, no human emissions of carbon to the atmosphere from the burning of fossil fuels.  In our model, the ECR is initially set to 0.005, but it can be altered as a graphical function of time to represent different policy scenarios.

## Global warming
The next step concerns the relationship between the accumulation of CO2 in the atmosphere and the global warming (The major part of future warming is projected to come from to CO2).

In [8]:
# CO2-Temperature slider
CO2_temp_factor = widgets.FloatSlider(min = 0,
                                 max = 0.3,
                                 value = 0.13, step = 0.01,
                                 description = 'CO2-Temperature factor',
                                 continuous_update=True,
                                 style = {'description_width': '175px'} ,layout={'width': '175px'},
                                 orientation='vertical')
CO2_temp_factor.observe(update_figure,names = 'value')

## Climate damage
Climate Damages are the costs associated with rising global temperatures. The way the model is set up, a 2°C increase in Global Temperature results in damages equal to 2.4% of Gross Output, but this rises to 9.6% for a temperature increase of 4°C, and 21.6% of Gross Output for a 6°C increase. This relationship between temperature change and damage involves temperature raised to an exponent that is initially set at 2, but can be adjusted. 

In [9]:
# Climate damage slider
clim_damage_exp = widgets.FloatSlider(min = 0,
                                 max = 3,
                                 value = 2, step = 0.01,
                                 description = 'Climate damage exponent',
                                 continuous_update=True,
                                 style = {'description_width': '175px'} ,layout={'width': '175px'},
                                 orientation='vertical')
clim_damage_exp.observe(update_figure,names = 'value')

## Relative Climate Costs
It will be useful to have a way of comparing the climate costs, which is the sum of the Abatement Costs and the Climate Damages in a relative sense so that we see what the percentage of these costs is relative to the Gross Output of the economy. The model includes this relative measure of the climate costs as follows:

        Relative Climate Costs = (Abatement Costs + Climate Damages)/Gross Output * 100

## Interactive figure
#### Initial simulation

In [10]:
population, prod, ec_cost, temp, rel_clim_damage = update_sim(pop_param, 
                                                          prod_grow_rate, 
                                                          ec_grow_rate, 
                                                          CO2_temp_factor, 
                                                          clim_damage_exp)

#### Figure of the results: world population, temperature and climate cost

In [11]:
pop_trace  = go.Scatter(x=time_range, y=population, name='population', marker_color = 'blue')
temp_trace = go.Scatter(x=time_range, y=temp, name='temp above preindustrial', marker_color = 'red',xaxis = "x", yaxis ="y2")
clim_trace = go.Scatter(x=time_range, y=rel_clim_damage, name='relative climate cost', marker_color = 'brown',xaxis = "x", yaxis ="y3")

layout = go.Layout(height=700,
    yaxis  = dict(range = [6000,12000],title = 'millions',domain=[0,0.30]),
    yaxis2 = dict(range = [0,10],title = 'degC',domain=[0.35,0.65]),
    yaxis3 = dict(range = [0,100],title = '%',domain=[0.7,1]))

fig = go.FigureWidget(data = [clim_trace,temp_trace,pop_trace], layout = layout)

### Plot the interactive figure

In [12]:
widgets.VBox([fig,widgets.HBox([pop_param,prod_grow_rate,ec_grow_rate,CO2_temp_factor,clim_damage_exp])])


    'data': [{'marker': {'color': 'brown'},
              'name': 'relative clim…